In [2]:
import numpy as np
import pandas as pd

In [3]:
#read in; had to use converters b/c the truck names had mixed types
df = pd.read_csv("20180101.csv", parse_dates=['date_fixed'], converters={'truck_name': str})

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39784 entries, 0 to 39783
Data columns (total 5 columns):
truck_name    39784 non-null object
date_fixed    39784 non-null datetime64[ns]
address       38861 non-null object
longitude     39784 non-null float64
latitude      39784 non-null float64
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 1.5+ MB


### Dates are Pandas Timestamps

In [28]:
df.loc[1700,'date_fixed']

Timestamp('2018-01-01 16:25:29')

### Use Timedelta method to shift time

In [29]:
df.loc[1700,'date_fixed'] - pd.Timedelta(hours=5)

Timestamp('2018-01-01 11:25:29')

In [6]:
#stripping white space from the truck names
df['truck_name'] = df['truck_name'].str.strip()

In [11]:
#sorted by truck then date/time
df = df.sort_values(by=['truck_name','date_fixed'],na_position='first')

In [30]:
df['date_fixed'] = df['date_fixed'] - pd.Timedelta(hours=5)

In [28]:
#test subset 
df4= df3[15000:15200]

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39784 entries, 8427 to 34783
Data columns (total 5 columns):
truck_name    39784 non-null object
date_fixed    39784 non-null datetime64[ns]
address       38861 non-null object
longitude     39784 non-null float64
latitude      39784 non-null float64
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 3.1+ MB


In [36]:
df[1700:1800]

,truck_name,date_fixed,address,longitude,latitude
204,178,2018-01-01 23:43:42,1392 ERIE BLVD E SYRACUSE 13210,-76.127819,43.050415
307,178,2018-01-01 23:43:54,1401-23 ERIE BLVD E & BEECH ST N,-76.126873,43.050237
408,178,2018-01-01 23:44:31,1636 ERIE BLVD E & COLUMBUS AV,-76.121783,43.049313
512,178,2018-01-01 23:44:49,1710 ERIE BLVD E,-76.120847,43.049439
539,178,2018-01-01 23:45:34,1915 ERIE BLVD E & PEAT ST & K,-76.115285,43.051184
542,178,2018-01-01 23:45:48,1915 ERIE BLVD E & PEAT ST & K,-76.115186,43.051675
546,178,2018-01-01 23:46:25,338 PEAT ST & CANAL ST,-76.115314,43.054301
550,178,2018-01-01 23:46:33,340 PEAT ST,-76.114919,43.054249
554,178,2018-01-01 23:46:45,418 PEAT ST REAR,-76.114109,43.054127
558,178,2018-01-01 23:47:43,1200 CANAL ST,-76.109290,43.055286


In [37]:
# get the manhattan distance between consecutive data points
df['manh_dist'] = abs(df.longitude - df.longitude.shift()) + abs(df.latitude - df.latitude.shift())

In [42]:
df[df.manh_dist > .01].count()

truck_name    239
date_fixed    239
address       236
longitude     239
latitude      239
manh_dist     239
dtype: int64

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39784 entries, 8427 to 34783
Data columns (total 6 columns):
truck_name    39784 non-null object
date_fixed    39784 non-null datetime64[ns]
address       38861 non-null object
longitude     39784 non-null float64
latitude      39784 non-null float64
manh_dist     39783 non-null float64
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 3.4+ MB


In [44]:
#how many data points per truck
df.groupby('truck_name').size()

truck_name
178     1722
179     1682
182        6
206      143
207        1
208        1
210     1285
211        1
212     1288
216     4065
251     3811
253        1
254       47
257        1
258      626
259       27
260.       1
261     3341
262        1
263     3272
264     2364
265     3461
266     2004
267        1
268      686
269        1
270        6
271     2398
282     2203
284     2234
285     2667
73       437
dtype: int64

# Left off here. Create an 'ID' column, which is a mapping from a point to a road segment

# Below here is not used

In [116]:
#NOT USED; generate test dataframe
test_row_iter = pd.DataFrame(np.random.randn(20,3),columns=list('ABC'))

In [118]:
#NOT USED
#for testing: set column C to absolute values
test_row_iter['C'] = abs(test_row_iter['C'])

In [85]:
#NOT USED
#how to iterate over rows and set a boolean value based on a conditional
for i in range(1,len(test_row_iter)):
    if test_row_iter.loc[i, 'C'] > 0:
        test_row_iter.loc[i,'D'] = True

In [120]:
#NOT USED
#set first boolean of the 'Keep' column to 'True', since we always want to keep the first data point
test_row_iter.loc[0,'Keep'] = True
#create a boolean column based on an the accumulated value of a column
threshhold = 2.
cum_dist = 0.
for i in range(1,len(test_row_iter)):
    if test_row_iter.loc[i, 'C'] + cum_dist > threshhold:
        test_row_iter.loc[i,'Keep'] = True
        cum_dist = 0
    else:
        test_row_iter.loc[i,'Keep'] = False
        cum_dist += test_row_iter.loc[i, 'C']
            

In [127]:
#NOT USED
test_row_iter = test_row_iter[test_row_iter.Keep == True]